Gradient Boosting Steps/Processes:
1. Initial prediction (Mean of the target)
2. Residual = Target - Prediction
    trees= []
3. For loop (loop for 1000's)
  4. Create a decision trees =>  tarin residual
  5. train(x, residuals)
  6. predicitons = predict(x) => predict residual
  7. Residual = Residual - learning_rate * prediction
  8. trees.append(tree)


1.Initial prediction => mean of target
2. REsidual = target- prediction
3. train decision tree = train residual
4. prediciton => prediciton of residual
5. new prediciton  = old prediciton + LR * prediciton
6.step 2 for all thre trees